In [1]:
directory = './EmoDB_dataset/wav'

# MFCC features

# to do: global normalisation

## Feature extraction

In [2]:
import os
import librosa
import numpy as np
from typing import Dict

def extract_mfcc_features(file_path: str, n_mfcc: int = 39, 
                          frame_size: float = 0.025, frame_stride: float = 0.01, 
                          n_segments: int = 10) -> np.ndarray:
    """
    Extracts 39 MFCC features framewise from an audio file and then applies
    average pooling to condense the features over time into an n x 39 feature matrix.
    
    Parameters:
      file_path (str): Path to the audio file.
      n_mfcc (int): Number of MFCC features to extract. Default is 39.
      frame_size (float): Length of each frame in seconds. Default is 0.025.
      frame_stride (float): Step between successive frames in seconds. Default is 0.01.
      n_segments (int): Number of segments (n) to pool the frames into.
    
    Returns:
      np.ndarray: A n x 39 array where each row is the average MFCC vector for that segment.
    """
    try:
        signal, sample_rate = librosa.load(file_path, sr=None)
        frame_length = int(frame_size * sample_rate)
        hop_length = int(frame_stride * sample_rate)
        
        # Extract MFCC features; result shape is (n_mfcc, T) where T is number of frames.
        mfcc = librosa.feature.mfcc(y=signal, sr=sample_rate, n_mfcc=n_mfcc,
                                    n_fft=frame_length, hop_length=hop_length)
        
        # Normalize the MFCC features along each coefficient dimension.
        mfcc_normalized = mfcc - np.mean(mfcc, axis=1, keepdims=True)
        
        # Transpose to shape (T, n_mfcc) for pooling along the time axis.
        mfcc_normalized = mfcc_normalized.T
        
        # Divide the frames into n_segments segments and compute the average for each segment.
        segments = np.array_split(mfcc_normalized, n_segments, axis=0)
        pooled_features = np.array([np.mean(seg, axis=0) for seg in segments])
        
        return pooled_features  # Shape: (n_segments, 39)
    
    except Exception as e:
        print(f"Error processing file {file_path}: {e}")
    return np.array([])

def process_directory_mfcc(directory: str, n_segments: int = 10) -> Dict[str, np.ndarray]:
    """
    Processes all .wav files in the given directory, extracting their MFCC features
    using average pooling to produce an n x 39 feature matrix for each file.
    
    Parameters:
      directory (str): Path to the directory containing .wav files.
      n_segments (int): Number of segments to pool the frames into for each file.
    
    Returns:
      Dict[str, np.ndarray]: A dictionary mapping filenames to their corresponding feature matrices.
    """
    feature_vectors = {}
    for filename in os.listdir(directory):
        if filename.endswith('.wav'):
            file_path = os.path.join(directory, filename)
            features = extract_mfcc_features(file_path, n_segments=n_segments)
            if features.size > 0:
                feature_vectors[filename] = features
    return feature_vectors


## Classifier on MFCC

In [3]:
import os
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.mixture import GaussianMixture
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report


def load_labels(csv_file: str) -> pd.DataFrame:
    return pd.read_csv(csv_file)

def prepare_dataset(features: dict, labels: pd.DataFrame):
    """
    Constructs the dataset by matching each audio file's feature matrix with its label.
    Since each file is represented as an n x 39 matrix (n segments by 39 features),
    we flatten it into a 1D feature vector of length n*39.
    """
    X = []
    y = []
    for _, row in labels.iterrows():
        file_id = row['Filename']
        if file_id in features:
            # Flatten the (n, 39) matrix to a 1D vector (n*39,)
            feature_matrix = features[file_id]
            feature_vector = feature_matrix.flatten()
            X.append(feature_vector)
            y.append(int(row['EmotionNumeric']))
    return np.array(X), np.array(y)


def train_and_evaluate(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # GMM Classifier
    gmm_model = GaussianMixture(n_components=len(np.unique(y)), random_state=42)
    gmm_model.fit(X_train)
    gmm_predictions = gmm_model.predict(X_test)

    gmm_acc = accuracy_score(y_test, gmm_predictions)
    print("GMM Classifier Report:")
    print(classification_report(y_test, gmm_predictions))

    # SVM Classifier
    svm_model = SVC(kernel='linear', random_state=42)
    svm_model.fit(X_train, y_train)
    svm_predictions = svm_model.predict(X_test)

    svm_acc = accuracy_score(y_test, svm_predictions)
    print("SVM Classifier Report:")
    print(classification_report(y_test, svm_predictions))

    return gmm_acc, svm_acc


# Wandb

In [4]:
import wandb
# import numpy as np

# def sweep_run(config=None):
#     with wandb.init(config=config):
#         config = wandb.config

#         # 1. Read the current sweep value of n_mfcc
#         n = config.n_mfcc
#         mfccFeatures = process_directory_mfcc(directory, n)

#         # 2. Load labels
#         labels_csv_path = "EmoDB_dataset/emotion_mapping_detailed.csv"
#         labels = load_labels(labels_csv_path)

#         # 3. Prepare dataset
#         X, y = prepare_dataset(mfccFeatures, labels)

#         # 4. Train both GMM and SVM
#         gmm_acc, svm_acc = train_and_evaluate(X, y)

#         # 5. Log SCALAR values — not lists
#         wandb.log({
#             "n_mfcc": n,
#             "gmm_accuracy": gmm_acc,
#             "svm_accuracy": svm_acc,
#             "feature_dim": X.shape[1]
#         })


# # Sweep config
# sweep_config = {
#     'method': 'grid',
#     'metric': {
#         'name': 'accuracies',  # You can also choose 'gmm_accuracies'
#         'goal': 'maximize'
#     },
#     'parameters': {
#         'n_mfcc': {
#             'values': np.arange(10, 120, 5).tolist()
#         }
#     }
# }

# # Launch the sweep
# sweep_id = wandb.sweep(sweep_config, project='ssp-mfcc-tuning-WithoutSilenceRemoval')
# wandb.agent(sweep_id, function=sweep_run)



In [6]:
import wandb
import numpy as np

def single_run_all_nmfccs():
    wandb.init(
        project="ssp-mfcc-tuning-WithoutSilenceRemoval",
        name="n_mfcc_sweep_single_run",
        config={}
    )

    for n in np.arange(1, 100, 5):
        mfccFeatures = process_directory_mfcc(directory, n)
        labels = load_labels("EmoDB_dataset/emotion_mapping_detailed.csv")
        X, y = prepare_dataset(mfccFeatures, labels)
        gmm_acc, svm_acc = train_and_evaluate(X, y)

        # Log scalars at each step
        wandb.log({
            "n_mfcc": n,
            "svm_accuracy": svm_acc,
            "gmm_accuracy": gmm_acc,
            "feature_dim": X.shape[1]
        })

    wandb.finish()

# Run it directly
single_run_all_nmfccs()


c:\Users\sanni\miniconda3\envs\coding2\lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(
c:\Users\sanni\miniconda3\envs\coding2\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\sanni\miniconda3\envs\coding2\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\sanni\miniconda3\envs

GMM Classifier Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        18
           1       0.19      1.00      0.31        20
           2       0.00      0.00      0.00        12
           3       0.00      0.00      0.00        14
           4       0.00      0.00      0.00        18
           5       0.00      0.00      0.00         9
           6       0.00      0.00      0.00        16

    accuracy                           0.19       107
   macro avg       0.03      0.14      0.04       107
weighted avg       0.03      0.19      0.06       107

SVM Classifier Report:
              precision    recall  f1-score   support

           0       0.17      1.00      0.29        18
           1       0.00      0.00      0.00        20
           2       0.00      0.00      0.00        12
           3       0.00      0.00      0.00        14
           4       0.00      0.00      0.00        18
           5       0.00      0.0

c:\Users\sanni\miniconda3\envs\coding2\lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


GMM Classifier Report:
              precision    recall  f1-score   support

           0       0.13      0.11      0.12        18
           1       0.00      0.00      0.00        20
           2       0.00      0.00      0.00        12
           3       0.20      0.29      0.24        14
           4       0.25      0.06      0.09        18
           5       0.08      0.44      0.14         9
           6       0.08      0.06      0.07        16

    accuracy                           0.11       107
   macro avg       0.11      0.14      0.09       107
weighted avg       0.11      0.11      0.09       107

SVM Classifier Report:
              precision    recall  f1-score   support

           0       0.43      0.67      0.52        18
           1       0.83      0.75      0.79        20
           2       0.29      0.17      0.21        12
           3       0.36      0.36      0.36        14
           4       0.62      0.56      0.59        18
           5       0.60      0.6

c:\Users\sanni\miniconda3\envs\coding2\lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


...

GMM Classifier Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        18
           1       0.17      0.05      0.08        20
           2       0.00      0.00      0.00        12
           3       0.14      0.71      0.23        14
           4       0.00      0.00      0.00        18
           5       0.00      0.00      0.00         9
           6       0.33      0.19      0.24        16

    accuracy                           0.13       107
   macro avg       0.09      0.14      0.08       107
weighted avg       0.10      0.13      0.08       107

SVM Classifier Report:
              precision    recall  f1-score   support

           0       0.32      0.50      0.39        18
           1       0.57      0.60      0.59        20
           2       0.17      0.08      0.11        12
           3       0.27      0.21      0.24        14
           4       0.50      0.39      0.44        18
           5       0.60      0.6

c:\Users\sanni\miniconda3\envs\coding2\lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


GMM Classifier Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        18
           1       0.00      0.00      0.00        20
           2       0.13      0.33      0.19        12
           3       0.12      0.36      0.18        14
           4       0.18      0.11      0.14        18
           5       0.00      0.00      0.00         9
           6       0.12      0.12      0.12        16

    accuracy                           0.12       107
   macro avg       0.08      0.13      0.09       107
weighted avg       0.08      0.12      0.09       107

SVM Classifier Report:
              precision    recall  f1-score   support

           0       0.37      0.61      0.46        18
           1       0.67      0.50      0.57        20
           2       0.17      0.08      0.11        12
           3       0.25      0.29      0.27        14
           4       0.71      0.28      0.40        18
           5       0.64      0.7

wandb: ERROR The nbformat package was not found. It is required to save notebook history.


feature_dim,▁▁▂▂▂▃▃▃▃▄▄▄▅▅▅▆▆▆▆▇▇▇██
gmm_accuracy,█▂▇▇▄▂▁▂█▅▄▂▂▂▅▇▆▆▁▅▂▇▄▃
n_mfcc,▁▁▂▂▂▃▃▃▃▄▄▄▅▅▅▆▆▆▆▇▇▇██
svm_accuracy,▁█▇▇▇▇▇█▇▇▆▇▇▆▇▇▆█▆▇▅▆▆▆
feature_dim,4524
gmm_accuracy,0.1215
n_mfcc,116
svm_accuracy,0.43925
